# Pynq-Z1 HDMI Sobel Filter Demo

* Signal Chain: HDMI In -> RGB-to-Grayscale -> Sobel Filter -> HDMI Out
* HDMI In/Out Resolution: 1080x1920 with 24bit per pixel
* Frame Rate: 32 frames per second
* Repository: https://github.com/beja65536/pz1_sobelfilter
* Based-on: http://www.wiki.xilinx.com/SDSoC+designs+and+platforms

### Step 1: Install package

In [ ]:
!sudo -H pip install --upgrade 'https://github.com/beja65536/pz1_sobelfilter'

### Step 2: Initialize Sobel filter and HDMI I/O

In [1]:
from pz1sofi import pz1sofi
hw = pz1sofi()
from pynq.drivers.video import HDMI
hdmi_out = HDMI('out')
hdmi_in = HDMI('in', init_timeout=10)
hdmi_in.start()
hdmi_out.start()

### Step 3: Run OpenCV Sobel Filter

In [ ]:
import numpy as np
import time
import cv2
num_frames = 10
time1 = time.time()
for i in range (num_frames):    
    np_frame = (np.frombuffer(hdmi_in.frame_raw(), 
                             dtype=np.uint8)).reshape(1080,1920,3)
    grad_x = cv2.Sobel(np_frame,cv2.CV_8U,1,0,ksize = 3, scale = 1, 
                       delta = 0,borderType = cv2.BORDER_DEFAULT)
    grad_y = cv2.Sobel(np_frame,cv2.CV_8U,0,1,ksize = 3, scale = 1, 
                       delta = 0, borderType = cv2.BORDER_DEFAULT)
    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)
    sobel_frame = cv2.addWeighted(abs_grad_x,0.5,abs_grad_y,0.5,0)
    hdmi_out.frame_raw(bytearray(sobel_frame))
time2 = time.time()
print("OpenCV: {0:.2f} fps".format(num_frames/(time2-time1)))

### Step 4: Image Processing with hardware-accelerated Sobel filter

In [2]:
fps = hw.sobel_filter(hw.get_frame_ptr(hdmi_in), hw.get_frame_ptr(hdmi_out),
         num_frames=20, get_fps=True)
print("SDSoC: {0:.2f} fps".format(fps))

SDSoC: 31.95 fps


### Step 5: Release Resources

In [ ]:
del edge_detect
hdmi_out.stop()
hdmi_in.stop()
del hdmi_in
del hdmi_out